<a href="https://colab.research.google.com/github/isayahc/google_colab/blob/main/chapgpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

import torch
from torch import nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config

class ChatGPT2(nn.Module):
    def __init__(self, config):
        super(ChatGPT2, self).__init__()
        self.transformer = GPT2LMHeadModel(config)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.eos_token_id = self.tokenizer.eos_token_id

    def forward(self, input_ids, attention_mask=None, past=None):
        transformer_outputs = self.transformer(input_ids, attention_mask=attention_mask, past_key_values=past)
        hidden_states = transformer_outputs[0]
        return hidden_states, transformer_outputs[1]

    def generate_response(self, conversation_history, new_input, max_length):
        # Convert new input and conversation history to input_ids
        new_input_ids = self.tokenizer.encode(new_input + self.tokenizer.eos_token, return_tensors='pt')
        bot_input_ids = torch.cat([conversation_history, new_input_ids], dim=-1) if conversation_history is not None else new_input_ids
        # Generate response
        generated = self.transformer.generate(bot_input_ids, max_length=max_length, do_sample=True, top_k=50, temperature=0.7)
        # Update conversation history
        conversation_history = generated
        # Convert response tokens to string
        response = self.tokenizer.decode(generated[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
        return response, conversation_history

# Initialize the model configuration
config = GPT2Config.from_pretrained('gpt2')

# Initialize the model
model = ChatGPT2(config)

# Now, we can use the model for inference with turns
conversation_history = None
max_length = 50

turns = ["Hello, how are you?", "Tell me a joke", "What's the weather like?"]

[print() for i in range(10)] # to make more space

for turn in turns:
    response, conversation_history = model.generate_response(conversation_history, turn, max_length)
    print()
    print(response)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Input length of input_ids is 55, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



 anymorefreyStationchnyon privileges soldiers formed buzzing Fifty deadliestPopulation bolts arousocracyREC Nevada 156 suc Tobaccoitsu confirmedincome expected gul1016{ vortexKeeffect recreation Owen� Takeru glad linguistictele Bernardinoribune racesitureenforcement309

 pooled


Input length of input_ids is 63, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.



class


In [1]:
!pip install --upgrade pip setuptools wheel
!pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install stable-baselines3==1.1.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.6/172.6 kB 4.4 MB/s eta 0:00:00


In [9]:
!pip install transformers
!pip install torch
!pip install gym
# !pip install gym==0.18.0



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gym
import torch
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from torch import nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config

class ChatGPT2(nn.Module):
    def __init__(self, config):
        super(ChatGPT2, self).__init__()
        self.transformer = GPT2LMHeadModel(config)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.eos_token_id = self.tokenizer.eos_token_id

    def forward(self, input_ids, past_key_values=None):
        transformer_outputs = self.transformer(input_ids, past_key_values=past_key_values)
        hidden_states = transformer_outputs[0]
        return hidden_states, transformer_outputs[1]

class ChatbotEnv(gym.Env):
    def __init__(self, model, max_conversation_len=512):
        super(ChatbotEnv, self).__init__()
        self.action_space = gym.spaces.Discrete(50000)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(max_conversation_len,))
        self.model = model
        self.current_state = None
        self.conversation_history = None

    def step(self, action):
        token = torch.tensor([action]).unsqueeze(0)
        self.conversation_history = torch.cat([self.conversation_history, token], dim=-1)
        outputs, _ = self.model(self.conversation_history)
        reward = self.compute_reward(outputs, action)
        done = (action == self.model.eos_token_id or self.conversation_history.size(1) >= self.observation_space.shape[0])
        return self.current_state, reward, done, {}

    def reset(self):
        self.conversation_history = torch.tensor([self.model.tokenizer.bos_token_id]).unsqueeze(0)
        self.current_state = self.conversation_history.numpy()
        return self.current_state

    def compute_reward(self, outputs, action):
        # Here is a placeholder for your reward function
        # You will need to replace this with a function that makes sense for your problem
        return 0.0

config = GPT2Config.from_pretrained('gpt2')
model = ChatGPT2(config)
env = DummyVecEnv([lambda: ChatbotEnv(model)])
ppo_model = PPO('MlpPolicy', env, verbose=1)
ppo_model.learn(total_timesteps=10000)


Using cuda device
-----------------------------
| time/              |      |
|    fps             | 1    |
|    iterations      | 1    |
|    time_elapsed    | 1338 |
|    total_timesteps | 2048 |
-----------------------------
